In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel('Online Retail.xlsx', sheet_name='Online Retail')
df.head(3)

InvoiceNo StockCode                         Description  Quantity  \
0    536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                 WHITE METAL LANTERN         6   
2    536365    84406B      CREAM CUPID HEARTS COAT HANGER         8   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom

In [3]:
df.shape

(541909, 8)

In [4]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
#Dropping customer ID column
df.drop('CustomerID',axis=1, inplace=True)

#Dropping rows with missing description
df.dropna(axis = 0, subset =['Description'], inplace = True)

In [6]:
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
Country        0
dtype: int64

In [7]:
df.shape

(540455, 7)

In [8]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [9]:
from mlxtend.frequent_patterns import apriori, association_rules

In [10]:
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [11]:
#Getting rid of spaces in description column
df['Description'] = df['Description'].str.strip()
df.head(4)

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   

          InvoiceDate  UnitPrice         Country  
0 2010-12-01 08:26:00       2.55  United Kingdom  
1 2010-12-01 08:26:00       3.39  United Kingdom  
2 2010-12-01 08:26:00       2.75  United Kingdom  
3 2010-12-01 08:26:00       3.39  United Kingdom

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
sales_UK = (df[df['Country'] == 'United Kingdom'].groupby(['InvoiceNo','Description'])['Quantity'].sum().
             unstack().reset_index().fillna(0).set_index('InvoiceNo'))

sales_Frane = (df[df['Country'] == 'France'].groupby(['InvoiceNo','Description'])['Quantity'].sum().
             unstack().reset_index().fillna(0).set_index('InvoiceNo'))

In [43]:
#Encoding data based on quantity
def encode(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

UK = sales_UK.applymap(encode)
France = sales_Frane.applymap(encode)

In [44]:
sales_Frane = France
sales_UK = UK

In [33]:
#Checking Association for France
freq_product = apriori(sales_Frane, min_support=0.03, use_colnames=True)

rules = association_rules(freq_product, metric='lift', min_threshold=1)
rules = rules.sort_values(['confidence','lift'], ascending = [False, False])
print(rules.head())

                                            antecedents  \
1224  (PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...   
794   (PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...   
1219  (PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...   
1196  (MINI PAINT SET VINTAGE, SET/6 RED SPOTTY PAPE...   
1192  (SET/20 RED RETROSPOT PAPER NAPKINS, MINI PAIN...   

                                            consequents  antecedent support  \
1224  (PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...            0.030369   
794                      (PACK OF 6 SKULL PAPER PLATES)            0.030369   
1219                     (PACK OF 6 SKULL PAPER PLATES)            0.030369   
1196  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...            0.030369   
1192  (SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...            0.030369   

      consequent support   support  confidence       lift  leverage  \
1224            0.034707  0.030369         1.0  28.812500  0.029315   
794             0.047722  0.

In [34]:
#Checking Association for UK
freq_product = apriori(sales_UK, min_support=0.03, use_colnames=True)

rules = association_rules(freq_product, metric='lift', min_threshold=1)
rules = rules.sort_values(['confidence','lift'], ascending = [False, False])
print(rules.head())

                         antecedents                        consequents  \
0  (GREEN REGENCY TEACUP AND SAUCER)  (ROSES REGENCY TEACUP AND SAUCER)   
1  (ROSES REGENCY TEACUP AND SAUCER)  (GREEN REGENCY TEACUP AND SAUCER)   
2          (JUMBO BAG PINK POLKADOT)          (JUMBO BAG RED RETROSPOT)   
4           (JUMBO STORAGE BAG SUKI)          (JUMBO BAG RED RETROSPOT)   
3          (JUMBO BAG RED RETROSPOT)          (JUMBO BAG PINK POLKADOT)   

   antecedent support  consequent support   support  confidence       lift  \
0            0.042379            0.043423  0.031807    0.750535  17.284272   
1            0.043423            0.042379  0.031807    0.732497  17.284272   
2            0.052589            0.087935  0.035619    0.677308   7.702369   
4            0.051273            0.087935  0.031671    0.617699   7.024495   
3            0.087935            0.052589  0.035619    0.405057   7.702369   

   leverage  conviction  zhangs_metric  
0  0.029967    3.834519       0.983838 

In [46]:
sales_Germany = (df[df['Country'] == 'Germany'].groupby(['InvoiceNo','Description'])['Quantity'].sum().
             unstack().reset_index().fillna(0).set_index('InvoiceNo'))

In [48]:
German = sales_Germany.applymap(encode)
sales_Germany = German

In [49]:
#Checking Association for Germany
freq_product = apriori(sales_Germany, min_support=0.03, use_colnames=True)

rules = association_rules(freq_product, metric='lift', min_threshold=1)
rules = rules.sort_values(['confidence','lift'], ascending = [False, False])
print(rules.head())

                           antecedents consequents  antecedent support  \
8              (BLUE HARMONICA IN BOX)   (POSTAGE)            0.036484   
83             (RED  HARMONICA IN BOX)   (POSTAGE)            0.034826   
127           (WOODLAND MINI BACKPACK)   (POSTAGE)            0.033167   
73         (PLASTERS IN TIN STRONGMAN)   (POSTAGE)            0.053068   
96   (RETROSPOT TEA SET CERAMIC 11 PC)   (POSTAGE)            0.043118   

     consequent support   support  confidence      lift  leverage  conviction  \
8              0.620232  0.036484    1.000000  1.612299  0.013856         inf   
83             0.620232  0.034826    1.000000  1.612299  0.013226         inf   
127            0.620232  0.033167    1.000000  1.612299  0.012596         inf   
73             0.620232  0.051410    0.968750  1.561915  0.018495   12.152570   
96             0.620232  0.041459    0.961538  1.550288  0.014716    9.873964   

     zhangs_metric  
8         0.394148  
83        0.393471  
127  